In [13]:
# load dataset, reshape and save to a new file
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
# define location of dataset
folder = "C:\\Users\\nidhimh\\Documents\\Occlusion\\occludedornot\\traincropfiltered\\"
photos, labels = list(), list()
# enumerate files in the directory
for file in listdir(folder):
    # determine class
    output = 0.0
    if file.startswith('occluded'):
        output = 1.0
    # load image
    photo = load_img(folder + file, target_size=(200, 200))
    # convert to numpy array
    photo = img_to_array(photo)
    # store
    photos.append(photo)
    labels.append(output)
# convert to a numpy arrays
photos = asarray(photos)
labels = asarray(labels)
print(photos.shape, labels.shape)
# save the reshaped photos
save('dataset_photos.npy', photos)
save('dataset_labels.npy', labels)

(2014, 200, 200, 3) (2014,)


In [14]:
# organize dataset into a useful structure
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random

dataset_home = "C:\\Users\\nidhimh\\Documents\\Occlusion\\occludedornot\\traincropfiltered\\"
subdirs = ['train/', 'test/']
for subdir in subdirs:
    # create label subdirectories
    labeldirs = ['notoccluded/', 'occluded/']
    for labldir in labeldirs:
        newdir = dataset_home + subdir + labldir
        makedirs(newdir, exist_ok=True)

In [4]:
# organize dataset into a useful structure
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random
# seed random number generator
seed(1)
# define ratio of pictures to use for validation
val_ratio = 0.25
# copy training dataset images into subdirectories
dataset_home = "C:\\Users\\nidhimh\\Documents\\Occlusion\\occludedornot\\filteredrefined\\"
src_directory = dataset_home
for file in listdir(src_directory):
    src = src_directory + '/' + file
    dst_dir = 'train/'
    if random() < val_ratio:
        dst_dir = 'test/'
    if file.startswith('occluded'):
        dst = dataset_home + dst_dir + 'occluded/'  + file
        copyfile(src, dst)
    elif file.startswith('notoccluded'):
        dst = dataset_home + dst_dir + 'notoccluded/'  + file
        copyfile(src, dst)

In [5]:
# baseline model for the dataset
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten, Dropout
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

# define cnn model
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# plot diagnostic learning curves
def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['acc'], color='blue', label='train')
    pyplot.plot(history.history['val_acc'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()

# run the test harness for evaluating a model
def run_test_harness():
    # define model
    model = define_model()
    # create data generator
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare iterators
    train_it = datagen.flow_from_directory('C:\\Users\\nidhimh\\Documents\\Occlusion\\occludedornot\\filteredrefined\\train\\',
        class_mode='binary', batch_size=64, target_size=(200, 200))
    test_it = datagen.flow_from_directory('C:\\Users\\nidhimh\\Documents\\Occlusion\\occludedornot\\filteredrefined\\test\\',
        class_mode='binary', batch_size=64, target_size=(200, 200))
    # fit model
    history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
        validation_data=test_it, validation_steps=len(test_it), epochs=25)
    # save model 
    model.save('occlusionstate.h5')
    model.save_weights('occlusion_weights.h5')
    # evaluate model
    _, acc = model.evaluate_generator(test_it, steps=len(test_it))
    print('> %.3f' % (acc * 100.0))
    # learning curves
    summarize_diagnostics(history)

# entry point, run the test harness
run_test_harness()

Found 1448 images belonging to 2 classes.
Found 459 images belonging to 2 classes.
Epoch 1/25
23/23 [==============================] - 82s 4s/step - loss: 0.7991 - acc: 0.6009 - val_loss: 0.6688 - val_acc: 0.6122
Epoch 2/25
23/23 [==============================] - 79s 3s/step - loss: 0.6253 - acc: 0.6714 - val_loss: 0.6593 - val_acc: 0.6340
Epoch 3/25
23/23 [==============================] - 78s 3s/step - loss: 0.6331 - acc: 0.6591 - val_loss: 0.6592 - val_acc: 0.6362
Epoch 4/25
23/23 [==============================] - 77s 3s/step - loss: 0.6209 - acc: 0.6780 - val_loss: 0.6562 - val_acc: 0.6601
Epoch 5/25
23/23 [==============================] - 77s 3s/step - loss: 0.6150 - acc: 0.6852 - val_loss: 0.6508 - val_acc: 0.6514
Epoch 6/25
23/23 [==============================] - 78s 3s/step - loss: 0.6134 - acc: 0.6913 - val_loss: 0.6478 - val_acc: 0.6754
Epoch 7/25
23/23 [==============================] - 77s 3s/step - loss: 0.6106 - acc: 0.6833 - val_loss: 0.6447 - val_acc: 0.6645
Epoch 8